# Fold Candidate Sequences with Boltz-2 (FRANKENCHAINS)

In [1]:
import pandas as pd
import os

In [2]:
## Read in candidates data
candidates_df = pd.read_excel('../candidates.xlsx', sheet_name='Antibody Candidates')
antigens_df = pd.read_excel('../candidates.xlsx', sheet_name='Antigens')

## Filter candidates based on Chotia test
candidates_df = candidates_df[candidates_df['test_chotia_pass'] == True]

## Get the antigen sequence for Nipah Glycoprotein G
antigen_seq = antigens_df.loc[antigens_df['antigen_id'] == 'nipah_gpG', 'antigen_sequence'].values[0]
print(f"Found {len(candidates_df)} Candidate Antibodies.\nUsing Antigen Sequence: {antigen_seq}")

Found 29 Candidate Antibodies.
Using Antigen Sequence: ICLQKTSNQILKPKLISYTLPVVGQSGTCITDPLLAMDEGYFAYSHLERIGSCSRGVSKQRIIGVGEVLDRGDEVPSLFMTNVWTPPNPNTVYHCSAVYNNEFYYVLCAVSTVGDPILNSTYWSGSLMMTRLAVKPKSNGGGYNQHQLALRSIEKGRYDKVMPYGPSGIKQGDTLYFPAVGFLVRTEFKYNDSNCPITKCQYSKPENCRLSMGIRPNSHYILRSGLLKYNLSDGENPKVVFIEISDQRLSIGSPSKIYDSLGQPVFYQASFSWDTMIKFGDVLTVNPLVVNWRNNTVISRPGQSQCPRFNTCPEICWEGVYNDAFLIDRINWISAGVFLDSNQTAENPVFTVFKDNEILYRAQLASEDTNAQKTITNCFLLKNKIWCISLVEIYDTGDNVIRPKLFAVKIPEQCTH


In [3]:
import yaml

## Define function to create Boltz-2 YAML configuration for multimer folding

def create_boltz_yaml(franken_chain: str, antigen_seq: str, output_path: str):
    config_json = {
        "version": 1,
        "sequences": [
            {
            "protein": {
                "id": "B",
                "sequence": franken_chain
            }
            },
            {
            "protein": {
                "id": "A",
                "sequence": antigen_seq
            }
            }
        ]
        }
    
    ## Make directories if they don't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    ## Write YAML file
    with open(output_path, 'w') as f:
        yaml.dump(config_json, f)

In [4]:
## Define a function for running Boltz-2 folding from the command line
def run_boltz_folding(yaml_config_path: str, output_dir: str, accelerator: str = 'cpu'):
    boltz_command = f"boltz predict {yaml_config_path} --out_dir {os.path.dirname(output_dir)} --use_msa_server --accelerator {accelerator}"

    os.system(boltz_command)

In [5]:
## Loop to create YAML files for Boltz-2 folding

for idx, row in candidates_df.iterrows():
    antibody_id = row['antibody_id']
    print(f"Generating YAML file for Candidate: {antibody_id}")
    
    output_dir = f'../data/candidates/structures_boltz2_frankenchain'
    yaml_config_path = f"{output_dir}/{antibody_id}.yaml"

    ## See if the yaml file for this antibody already exists
    if os.path.exists(yaml_config_path):
        print(f"\tYAML file for {antibody_id} already exists, skipping YAML creation.")
        continue

    ## Create YAML file
    create_boltz_yaml(row['franken_chain'], antigen_seq, yaml_config_path)

Generating YAML file for Candidate: sbio-nipahgpg-001
Generating YAML file for Candidate: sbio-nipahgpg-002
Generating YAML file for Candidate: sbio-nipahgpg-003
Generating YAML file for Candidate: sbio-nipahgpg-004
Generating YAML file for Candidate: sbio-nipahgpg-005
Generating YAML file for Candidate: sbio-nipahgpg-008
Generating YAML file for Candidate: sbio-nipahgpg-009
Generating YAML file for Candidate: sbio-nipahgpg-010
Generating YAML file for Candidate: sbio-nipahgpg-011
Generating YAML file for Candidate: sbio-nipahgpg-012
Generating YAML file for Candidate: sbio-nipahgpg-013
Generating YAML file for Candidate: sbio-nipahgpg-014
Generating YAML file for Candidate: sbio-nipahgpg-015
Generating YAML file for Candidate: sbio-nipahgpg-016
Generating YAML file for Candidate: sbio-nipahgpg-017
Generating YAML file for Candidate: sbio-nipahgpg-018
Generating YAML file for Candidate: sbio-nipahgpg-019
Generating YAML file for Candidate: sbio-nipahgpg-020
Generating YAML file for Can

In [6]:
## Loop to run Boltz-2 folding

for idx, row in candidates_df.iterrows():
    antibody_id = row['antibody_id']
    print(f"Folding Candidate: {antibody_id}")
    
    output_dir = f'../data/candidates/structures_boltz2_frankenchain'
    yaml_config_path = f"{output_dir}/{antibody_id}.yaml"

    ## See if the Boltz-2 directory for this antibody already exists
    if os.path.exists(f"{output_dir}/boltz_results_{antibody_id}"):
        print(f"\tOutput directory for {antibody_id} already exists, skipping folding.")
        continue

    ## Run Boltz folding
    run_boltz_folding(yaml_config_path, output_dir, accelerator='gpu')

Folding Candidate: sbio-nipahgpg-001
MSA server enabled: https://api.colabfold.com
MSA server authentication: no credentials provided
Checking input data.
Processing 1 inputs with 1 threads.


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]

Generating MSA for ../data/candidates/structures_boltz2_frankenchain/sbio-nipahgpg-001.yaml with 2 protein entities.
Calling MSA server for target sbio-nipahgpg-001 with 2 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server



PENDING:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0/300 [elapsed: 00:00 remaining: ?]

Running structure prediction for 1 input.


...

Generating MSA for ../data/candidates/structures_boltz2_frankenchain/sbio-nipahgpg-031.yaml with 2 protein entities.
Calling MSA server for target sbio-nipahgpg-031 with 2 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server



PENDING:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0/300 [elapsed: 00:00 remaining: ?]

Running structure prediction for 1 input.


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.0.post0, which is newer than your current Lightning version: v2.5.0
You are using a CUDA device ('NVIDIA H100 NVL') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████���██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00,  0.05it/s